# Zea mays functions

> Fill in a module description here

In [ ]:
#| default_exp zma

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import numpy  as np
import pandas as pd

import torch
import torch.nn.functional as F
from   torch.utils.data import Dataset


import sparsevnn
from   sparsevnn.core import\
    VNNHelper, \
    structured_layer_info, \
    SparseLinearCustom
from   sparsevnn.kegg import \
    kegg_connections_build, \
    kegg_connections_clean, \
    kegg_connections_append_y_hat, \
    kegg_connections_sanitize_names


import lightning.pytorch as pl

### From EnvDL

In [ ]:
# from EnvDL.dlfn import BigDataset, plDNN_general

In [ ]:
#| export
def np_3d_to_hilbert(
    in_seq, # This should be a 3d numpy array with dimensions of [samples, sequence, channels] 
    **kwargs
):
    "This is the 3d version of `np_2d_to_hilbert`. The goal is to process all of the samples of an array in one go."
    import numpy as np
    import tqdm
    from tqdm import tqdm
    
    import hilbertcurve
    from hilbertcurve.hilbertcurve import HilbertCurve

    import EnvDL
    from EnvDL.dna import calc_needed_hilbert_p
    
    n_snps = in_seq.shape[1]
    n_channels = in_seq.shape[-1]
    temp = in_seq

    p_needed = calc_needed_hilbert_p(n_needed=n_snps)
    
    # Data represented need not be continuous -- it need only have int positions
    # a sequence or a sequence with gaps can be encoded
    hilbert_curve = HilbertCurve(
        p = p_needed, # iterations i.e. hold 4^p positions
        n = 2    # dimensions
        )

    points = hilbert_curve.points_from_distances(range(n_snps))

    dim_0 = np.max(np.array(points)[:, 0])+1 # add 1 to account for 0 indexing
    dim_1 = np.max(np.array(points)[:, 1])+1
    temp_mat = np.zeros(shape = [in_seq.shape[0], dim_0, dim_1, n_channels])
    temp_mat[temp_mat == 0] = np.nan         #  empty values being used for visualization
    
    if "silent" in kwargs:
        for i in range(n_snps):
            temp_mat[:,                          # sample
                     points[i][0], points[i][1], # x, y
                     :] = temp[:, i]             # channels
    else:
        for i in tqdm(range(n_snps)):
            temp_mat[:,                          # sample
                     points[i][0], points[i][1], # x, y
                     :] = temp[:, i]             # channels

    return(temp_mat)

In [ ]:
#| export
class BigDataset(Dataset):
    def __init__(
        self,
        lookup_obs,
        lookups_are_filtered = False, # This is a critical piece of information. For deduplicated lookups to work they must either be filtered so that the 
                                      # lookup's length matchs y's length (idx will map to the right row) _or_ instead lookup_obs must contain a mapping 
                                      # from the y's current length to the row in the full dataset. 
                                      # If False idx -> lookup_obs[idx] -> lookup_env[idx] -> W[idx]
                                      # If True  idx ------------------ -> lookup_env[idx] -> W[idx]
#         lookup_geno,
#         lookup_env,
#         y,
#         G, 
#         G_type
#         S,
#         P,
#         W,
#         W_type,
#         send_batch_to_gpu # 'cuda:0' but '0' and 'cuda0' would also work
        transform = None, 
        target_transform = None,
        **kwargs 
        ):
        """
        This class produces a set with one or more input tensors. For flexibility the only _required_ input is `lookup_obs`, a tensor with the index of observations. 
        Everything else is provided as a kwarg. Output is a list of tensors1 ordered [y, G, S, W], any of these not initalized will be missing but not empty (e.g. [y, S, W] not [y, None, S, W]).       
        Used inputs are:
        lookup_obs: index for y, used by __getitem__ for obs_idx
        lookup_geno: index for G, row obs_idx, column 1 is geno_idx (geno information is deduplicated, hence the need for a lookup)
        lookup_env: index for S & W, , row obs_idx, column 1 is env_idx (env information is deduplicated, hence the need for a lookup)
        y: yield
        G: Genomic information 
        G_type: how the infomation should be returned, 'raw', 'hilbert', or 'list' (i.e. of tensors for snps in each gene)
        S: Soil information
        P: Planting/Harvest date contained in column 0, 1 respectively 
        W: Weather data
        W_type: how the infomation should be returned, 'raw' or 'hilbert'

        1 G may also be returned as a list of tensors
        """
        # Lookup info (so that deduplication works)
        self.lookup_obs = lookup_obs
        self.lookups_are_filtered = lookups_are_filtered
        # if 'lookup_obs'  in kwargs: self.lookup_obs  = kwargs['lookup_obs'];
        if 'lookup_geno' in kwargs: self.lookup_geno = kwargs['lookup_geno'];
        if 'lookup_env'  in kwargs: self.lookup_env  = kwargs['lookup_env'];
        # Data
        if 'y' in kwargs: self.y = kwargs['y'];
        if 'G' in kwargs: self.G = kwargs['G'];
        if 'S' in kwargs: self.S = kwargs['S'];
        if 'P' in kwargs: self.P = kwargs['P']; # PlantHarvest so that planting can be added into W
        if 'W' in kwargs: self.W = kwargs['W'];
        # Data prep state information
        if 'G_type' in kwargs: self.G_type = kwargs['G_type']; # raw, hilbert, list
        if 'W_type' in kwargs: self.W_type = kwargs['W_type']; # raw, hilbert
        # Data to be returned
        self.out_names = [e for e in ['y', 'G', 'S', 'W'] if e in kwargs]

        # Is data starting on the desired device or does it need to be cycled on and off?
        # This is based on ACGTDataset
        self.send_batch_to_gpu = None
        if 'send_batch_to_gpu' in kwargs.keys():
            send_batch_to_gpu = kwargs['send_batch_to_gpu']
            if type(send_batch_to_gpu) == str: 
                send_batch_to_gpu = send_batch_to_gpu.lower()
                if len(send_batch_to_gpu) >= 1:
                    # remove characters in 'cuda:'
                    send_batch_to_gpu = ''.join([e for e in send_batch_to_gpu if e not in ['c', 'u', 'd', 'a', ':']])
                self.send_batch_to_gpu = int(send_batch_to_gpu)
            elif type(send_batch_to_gpu) == int: 
                self.send_batch_to_gpu = send_batch_to_gpu
            else:
                print('send_batch_to_gpu kwarg ignored. Must be a string or int. Ideally of form "cuda:0"')

        # Transformations
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.lookup_obs)
    

    # These used to be in __getitem__ but separating them like this allows for them to be overwritten more easily
    def get_y(self, idx):
        y_idx = self.y[idx]
        if self.transform:
            y_idx = self.transform(y_idx)
        return(y_idx)
        
    def get_G(self, idx):
        geno_idx = self.lookup_geno[idx, 1]
        if self.G_type in ['raw', 'hilbert']:
            G_idx = self.G[geno_idx]
        if 'list' == self.G_type:
            G_idx = [e[geno_idx] for e in self.G]
        if self.transform:
            G_idx = self.transform(G_idx)
        return(G_idx)

    def get_S(self, idx):
        env_idx = self.lookup_env[idx, 1]
        S_idx = self.S[env_idx]
        if self.transform:
            S_idx = self.transform(S_idx)
        return(S_idx)

    def get_W(self, idx):
        W_device = torch.Tensor(self.W).get_device()

        env_idx = self.lookup_env[idx, 1]
        # get growing information
        WPlant = np.zeros(365)
        # WPlant[self.P[obs_idx, 0]:self.P[obs_idx, 1]] = 1
        WPlant[self.P[idx, 0]:self.P[idx, 1]] = 1
        if self.W_type == 'raw':
            WPlant = torch.from_numpy(WPlant).to(torch.float)
            # if needed send to gpu
            if W_device != -1: WPlant = WPlant.to(W_device)            
            W_idx = torch.concatenate([self.W[env_idx], WPlant[None, :]], axis = 0)
        if self.W_type == 'hilbert':
            # convert growing info to hilbert curve
            WPlant_hilb = np_3d_to_hilbert(WPlant[None, :, None], silent = True)
            WPlant_hilb = WPlant_hilb.squeeze(axis = 3)
            WPlant_hilb[np.isnan(WPlant_hilb)] = 0
            WPlant_hilb = torch.from_numpy(WPlant_hilb).to(torch.float)
            # if needed send to gpu
            if W_device != -1: WPlant_hilb = WPlant_hilb.to(W_device)
            W_idx = torch.concatenate([self.W[env_idx], WPlant_hilb], axis = 0)
        if self.transform:
            W_idx = self.transform(W_idx)
        return(W_idx)
    
    def __getitem__(self, idx):
        out = []
        obs_idx = idx
        if self.lookups_are_filtered == False:
            obs_idx = self.lookup_obs[idx]

        if 'y' in self.out_names: out += [self.get_y(obs_idx)]
        if 'G' in self.out_names: out += [self.get_G(obs_idx)]
        if 'S' in self.out_names: out += [self.get_S(obs_idx)]
        if 'W' in self.out_names: out += [self.get_W(obs_idx)]

        # send all to gpu    
        if self.send_batch_to_gpu is not None:
            out = [[ee.to(self.send_batch_to_gpu) for ee in e] if type(e)==list else e.to(self.send_batch_to_gpu) for e in out]     

        return out


In [ ]:
#| export
class plDNN_general(pl.LightningModule):
    def __init__(self, mod, log_weight_stats = False):
        super().__init__()
        self.mod = mod
        self.log_weight_stats = log_weight_stats
        
    def training_step(self, batch, batch_idx):
        y_i, x_i = batch
        pred = self.mod(x_i)
        loss = F.mse_loss(pred, y_i)
        self.log("train_loss", loss)
        
        if self.log_weight_stats:
            with torch.no_grad():
                weight_list=[(name, param) for name, param in self.mod.named_parameters() if name.split('.')[-1] == 'weight']
                for l in weight_list:
                    self.log(("train_mean"+l[0]), l[1].mean())
                    self.log(("train_std"+l[0]), l[1].std())        

        return(loss)
        
    def validation_step(self, batch, batch_idx):
        y_i, x_i = batch
        pred = self.mod(x_i)
        loss = F.mse_loss(pred, y_i)
        self.log('val_loss', loss)        
     
    def configure_optimizers(self, **kwargs):
        optimizer = torch.optim.Adam(self.parameters(), **kwargs)
        return optimizer    

In [ ]:
#| export
def mask_columns(df, # A dataframe containing the column to use for the mask
                col_name = 'Hybrid', # Column containing the values in `holdouts`
                holdouts = ['M0143/LH185', 'M0003/LH185'] # A list of values to match
                ):
    """Create a dataframe containing one mask or more mask for a list of `holdouts`."""
    out = [pd.DataFrame(df.loc[:, col_name] == holdout
            ).rename(columns = {col_name:holdout})
            for holdout in holdouts]
    
    out = pd.concat(out, axis=1)
    return out

In [ ]:
#| export
def mask_parents(
        df, # Dataframe containing a column with a genotype
        col_name = 'Hybrid', # The genotype column name
        holdout_parents = ['M0143'], # The genotype or genotypes that will be held out
        sep = '/' # Separator between parents. If not present (inbred genotype) that's okay.
    ):
    """Create a dataframe containing one mask or more based on a parent's genotype"""
    def  _mask_parent(df_FM, holdout = 'PHZ51'):
        holdout=   holdout.upper()
        mask_F = df_FM.F.str.upper() == holdout
        mask_M = df_FM.M.str.upper() == holdout
        mask = (mask_F | mask_M)
        return mask

    df[['F', 'M']] = df[col_name].str.split(sep, n=1, expand=True)
    mask = pd.concat([_mask_parent(df_FM=df, holdout=e) for e in holdout_parents], axis=1
            ).rename(columns={i:holdout_parents[i] for i in range(len(holdout_parents))})
    return mask

### vnnpaper specific

In [ ]:
#| export
def _dist_scale_function(out, dist, decay_rate):
    scale = 1/(1+decay_rate*dist)
    out = round(scale * out)
    out = max(1, out)
    return out

In [ ]:
#| export
def _expand_node_shortcut(vnn_helper, query = 'y_hat'):
    # define new entries
    if True in [True if e in vnn_helper.edge_dict.keys() else False for e in 
                [f'{query}_res_-2', f'{query}_res_-1']
                ]:
        print('Warning! New node name already exists! Overwriting existing node!')

    # Add residual connection in graph
    vnn_helper.edge_dict[f'{query}_res_-2'] = vnn_helper.edge_dict[query] 
    vnn_helper.edge_dict[f'{query}_res_-1'] = [f'{query}_res_-2']
    vnn_helper.edge_dict[query]             = [f'{query}_res_-2', f'{query}_res_-1']

    # Add new nodes, copying information from query node
    vnn_helper.node_props[f'{query}_res_-2'] = vnn_helper.node_props[query] 
    vnn_helper.node_props[f'{query}_res_-1'] = vnn_helper.node_props[query]

    return vnn_helper

In [ ]:
#| export

def vnn_factory_1(parsed_kegg_gene_entries, params, ACGT_gene_slice_list):

    print(''.join('#' for i in range(80)))
    print(params)
    print(''.join('#' for i in range(80)))
    
    
    default_out_nodes_inp  = params['default_out_nodes_inp' ]
    default_out_nodes_edge = params['default_out_nodes_edge'] 
    default_out_nodes_out  = params['default_out_nodes_out' ]

    default_drop_nodes_inp = params['default_drop_nodes_inp' ] 
    default_drop_nodes_edge= params['default_drop_nodes_edge'] 
    default_drop_nodes_out = params['default_drop_nodes_out' ] 

    default_reps_nodes_inp = params['default_reps_nodes_inp' ]
    default_reps_nodes_edge= params['default_reps_nodes_edge']
    default_reps_nodes_out = params['default_reps_nodes_out' ]



    default_decay_rate = params['default_decay_rate' ]



    # Clean up KEGG Pathways -------------------------------------------------------
    # Same setup as above to create kegg_gene_brite
    # Restrict to only those with pathway
    kegg_gene_brite = [e for e in parsed_kegg_gene_entries if 'BRITE' in e.keys()]

    # also require to have a non-empty path
    kegg_gene_brite = [e for e in kegg_gene_brite if not e['BRITE']['BRITE_PATHS'] == []]

    print('Retaining '+ str(round(len(kegg_gene_brite)/len(parsed_kegg_gene_entries), 4)*100)+'%, '+str(len(kegg_gene_brite)
        )+'/'+str(len(parsed_kegg_gene_entries)
        )+' Entries'
        )
    # kegg_gene_brite[1]['BRITE']['BRITE_PATHS']


    kegg_connections = kegg_connections_build(kegg_gene_brite = kegg_gene_brite, 
                                            n_genes = len(kegg_gene_brite)) 
    kegg_connections = kegg_connections_clean(         kegg_connections = kegg_connections)
    #TODO think about removing 
    # "Not Included In
    # Pathway Or Brite"
    # or reinstate 'Others'

    kegg_connections = kegg_connections_append_y_hat(  kegg_connections = kegg_connections)
    kegg_connections = kegg_connections_sanitize_names(kegg_connections = kegg_connections, 
                                                    replace_chars = {'.':'_'})


    # Initialize helper for input nodes --------------------------------------------
    myvnn = VNNHelper(edge_dict = kegg_connections)

    # Get a mapping of brite names to tensor list index
    find_names = myvnn.nodes_inp # e.g. ['100383860', '100278565', ... ]
    lookup_dict = {}

    # the only difference lookup_dict and brite_node_to_list_idx_dict above is that this is made using the full set of genes in the list 
    # whereas that is made using kegg_gene_brite which is a subset
    for i in range(len(parsed_kegg_gene_entries)):
        if 'BRITE' not in parsed_kegg_gene_entries[i].keys():
            pass
        elif parsed_kegg_gene_entries[i]['BRITE']['BRITE_PATHS'] == []:
            pass
        else:
            name = parsed_kegg_gene_entries[i]['BRITE']['BRITE_PATHS'][0][-1]
            if name in find_names:
                lookup_dict[name] = i
    # lookup_dict    

    brite_node_to_list_idx_dict = {}
    for i in range(len(kegg_gene_brite)):
        brite_node_to_list_idx_dict[str(kegg_gene_brite[i]['BRITE']['BRITE_PATHS'][0][-1])] = i        

    # Get the input sizes for the graph
    size_in_zip = zip(myvnn.nodes_inp, [np.prod(ACGT_gene_slice_list[lookup_dict[e]].shape[1:]) for e  in myvnn.nodes_inp])

    # Set node defaults ------------------------------------------------------------
    # init input node sizes
    myvnn.set_node_props(key = 'inp', node_val_zip = size_in_zip)

    # init node output sizes
    myvnn.set_node_props(key = 'out', node_val_zip = zip(myvnn.nodes_inp, [default_out_nodes_inp  for e in myvnn.nodes_inp]))
    myvnn.set_node_props(key = 'out', node_val_zip = zip(myvnn.nodes_edge,[default_out_nodes_edge for e in myvnn.nodes_edge]))
    myvnn.set_node_props(key = 'out', node_val_zip = zip(myvnn.nodes_out, [default_out_nodes_out  for e in myvnn.nodes_out]))

    # # options should be controlled by node_props
    myvnn.set_node_props(key = 'flatten', node_val_zip = zip(myvnn.nodes_inp, [True for e in myvnn.nodes_inp]))

    myvnn.set_node_props(key = 'reps', node_val_zip = zip(myvnn.nodes_inp, [default_reps_nodes_inp  for e in myvnn.nodes_inp]))
    myvnn.set_node_props(key = 'reps', node_val_zip = zip(myvnn.nodes_edge,[default_reps_nodes_edge for e in myvnn.nodes_edge]))
    myvnn.set_node_props(key = 'reps', node_val_zip = zip(myvnn.nodes_out, [default_reps_nodes_out  for e in myvnn.nodes_out]))

    myvnn.set_node_props(key = 'drop', node_val_zip = zip(myvnn.nodes_inp, [default_drop_nodes_inp  for e in myvnn.nodes_inp]))
    myvnn.set_node_props(key = 'drop', node_val_zip = zip(myvnn.nodes_edge,[default_drop_nodes_edge for e in myvnn.nodes_edge]))
    myvnn.set_node_props(key = 'drop', node_val_zip = zip(myvnn.nodes_out, [default_drop_nodes_out  for e in myvnn.nodes_out]))


    # Scale node outputs by distance -----------------------------------------------
    dist = sparsevnn.core.vertex_from_end(
        edge_dict = myvnn.edge_dict,
        end =myvnn.dependancy_order[-1]
    )

    # overwrite node outputs with a size inversely proportional to distance from prediction node
    for query in list(dist.keys()):
        myvnn.node_props[query]['out'] = _dist_scale_function(
            out = myvnn.node_props[query]['out'],
            dist = dist[query],
            decay_rate = default_decay_rate)
        

    # Expand out node replicates ---------------------------------------------------
    nodes = [node for node in myvnn.dependancy_order if myvnn.node_props[node]['reps'] > 1]

    node_expansion_dict = {
        node: [node if i==0 else f'{node}_{i}' for i in range(myvnn.node_props[node]['reps'])]
        for node in nodes}
    #   current       1st          2nd (new)      3rd (new)
    # {'100798274': ['100798274', '100798274_1', '100798274_2'], ...

    # the keys don't change here. The values will be updated and then new k:v will be inserted
    myvnn.edge_dict = {k:[e if e not in node_expansion_dict.keys() 
        else node_expansion_dict[e][-1]
        for e in myvnn.edge_dict[k] ] for k in myvnn.edge_dict}

    # now insert connectsion to new nodes: A -> A_rep_1 -> A_rep_2
    for node in node_expansion_dict:
        for pair in zip(node_expansion_dict[node][1:], node_expansion_dict[node]):
            myvnn.edge_dict[pair[0]] = [pair[1]]

    # now add those new nodes
    # create a new node for all the nodes
    for node in node_expansion_dict:
        for new_node in node_expansion_dict[node][1:]:
            myvnn.node_props[new_node] = {k:myvnn.node_props[node][k] for k in myvnn.node_props[node] if k != 'inp'}

    new_vnn = VNNHelper(edge_dict= myvnn.edge_dict)
    new_vnn.node_props = myvnn.node_props
    myvnn = new_vnn


    # init edge node input size (propagate forward input/edge outpus)
    myvnn.calc_edge_inp()

    # replace lookup so that it matches the lenght of the input tensors
    new_lookup_dict = {}
    for i in range(len(myvnn.nodes_inp)):
        new_lookup_dict[myvnn.nodes_inp[i]] = i
    
    return myvnn,  lookup_dict #new_lookup_dict


In [ ]:
#| export

def vnn_factory_2(vnn_helper, node_to_inp_num_dict):
    myvnn = vnn_helper

    node_props = myvnn.node_props
    # Linear_block = Linear_block_reps,
    edge_dict = myvnn.edge_dict
    dependancy_order = myvnn.dependancy_order
    # node_to_inp_num_dict = new_lookup_dict

    # Build dependancy dictionary --------------------------------------------------
    # check dep order
    tally = []
    for d in dependancy_order:
        if edge_dict[d] == []:
            tally.append(d)
        elif False not in [True if e in tally else False for e in edge_dict[d]]:
            tally.append(d)
        else:
            print('error!')
            break


    # build output nodes 
    d_out = {0:[]}
    for d in dependancy_order:
        if edge_dict[d] == []:
            d_out[min(d_out.keys())].append(d)
        else:
            # print((d, edge_dict[d]))

            d_out_i = 1+max(sum([[key for key in d_out.keys() if e in d_out[key]]
                    for e in edge_dict[d]], []))
            
            if d_out_i not in d_out.keys():
                d_out[d_out_i] = []
            d_out[d_out_i].append(d)


    # build input nodes NOPE. THE PASSHTROUGHS! 
    d_eye = {}
    tally = []
    for i in range(max(d_out.keys()), min(d_out.keys()), -1):
        # print(i)
        nodes_needed = sum([edge_dict[e] for e in d_out[i]], [])+tally
        # check against what is there and then dedupe
        nodes_needed = [e for e in nodes_needed if e not in d_out[i-1]]
        nodes_needed = list(set(nodes_needed))
        tally = nodes_needed
        d_eye[i] = nodes_needed

    # d_inp[0]= d_out[0]
    # [len(d_eye[i]) for i in d_eye.keys()]
    # [(key, len(d_out[key])) for key in d_out.keys()]


    dd = {}
    for i in d_eye.keys():
        dd[i] = {'out': d_out[i],
                'inp': d_out[i-1],
                'eye': d_eye[i]}
    # plus special 0 layer that handles the snps
        
    dd[0] = {'out': d_out[0],
            'inp': d_out[0],
            'eye': []}


    # check that the output nodes' inputs are satisfied by the same layer's inputs (inp and eye)
    for i in dd.keys():
        # out node in each
        for e in dd[i]['out']:
            # node depends in inp/eye
            node_pass_list = [True if ee in dd[i]['inp']+dd[i]['eye'] else False 
                            for ee in edge_dict[e]]
            if False not in node_pass_list:
                pass
            else:
                print('exit') 


    # print("Layer\t#In\t#Out")
    # for i in range(min(dd.keys()), max(dd.keys())+1, 1):
    #     node_in      = [node_props[e]['out'] for e in dd[i]['inp']+dd[i  ]['eye'] ]
    #     if i == max(dd.keys()):
    #         node_out = [node_props[e]['out'] for e in dd[i]['out'] ]
    #     else:
    #         node_out = [node_props[e]['out'] for e in dd[i]['out']+dd[i+1]['eye']]
    #     print(f'{i}:\t{sum(node_in)}\t{sum(node_out)}')

    M_list = [structured_layer_info(i = ii, node_groups = dd, node_props= node_props, edge_dict = edge_dict, as_sparse=True) for ii in range(0, max(dd.keys())+1)]
    return M_list


In [ ]:
#| export

def vnn_factory_3(M_list):
    layer_list = []
    for i in range(len(M_list)):
        
        apply_relu = None
        if i+1 != len(M_list): # apply relu to all but the last layer
            apply_relu = F.relu
        

        l = SparseLinearCustom(
            M_list[i].weight.shape[1], # have to transpose this?
            M_list[i].weight.shape[0],
            connectivity   = torch.LongTensor(M_list[i].weight.coalesce().indices()),
            custom_weights = M_list[i].weight.coalesce().values(), 
            custom_bias    = M_list[i].bias.clone().detach(), 
            weight_grad_bool = M_list[i].weight_grad_bool, 
            bias_grad_bool   = M_list[i].bias_grad_bool, #.to_sparse()#.indices()
            dropout_p        = M_list[i].dropout_p,
            nonlinear_transform= apply_relu
            )

        layer_list += [l]
        
    return layer_list

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

ValueError: not enough values to unpack (expected 2, got 1)